In [1]:
!pip install requests
!pip install BeautifulSoup4


import os
import re
import requests
from bs4 import BeautifulSoup
from pathlib import Path

#Définition des Urls 
urls={ "Ali Gatie" : "https://genius.com/Ali-gatie-its-you-lyrics",
       "We the Kings" : "https://genius.com/We-the-kings-sad-song-lyrics",
       "Eminem" : "https://genius.com/Eminem-lose-yourself-lyrics",
       "Hozier" : "https://genius.com/Hozier-take-me-to-church-lyrics",
       "Pharrell Williams" : "https://genius.com/Pharrell-williams-happy-lyrics",
       "OneRepublic" : "https://genius.com/Onerepublic-counting-stars-lyrics",
       "Miley Cyrus" : "https://genius.com/Miley-cyrus-wrecking-ball-lyrics",
       "Justin Timberlake" : "https://genius.com/Justin-timberlake-cant-stop-the-feeling-lyrics",
       "Ed Sheeran" : "https://genius.com/Ed-sheeran-perfect-lyrics",
       "Nicky Youre" : "https://genius.com/Nicky-youre-and-dazy-sunroof-lyrics"
}

#J'indique le chemin et si cela n'existe pas il va me le créer
base_path = Path.home()/ "PROJECT" / "data" / "clean" 
os.makedirs(base_path, exist_ok=True)

#Parcours les paires de nom et url et fais du scraping
for name, url in urls.items():
    print(f"Scraping {name} depuis {url}...")
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Erreur {response.status_code} pour {url}")
            continue

        #Parse le contenu HTML
        soup = BeautifulSoup(response.text,"html.parser")
        
        #Récupération des div avec les paroles
        containers = soup.find_all("div", class_=lambda x: x and "Lyrics__Container" in x)
        
        #Nettoie le corpus, decoupe en ligne selon et saut de ligne et les mots reccurents
        ligne = []
        for div in containers:
            for line in div.get_text(separator="\n").split("\n"):
                if len(line.split()) >=3 and not any(x in line.lower() for x in ["lyrics", "read more", "translations", "contributors"]):
                    ligne.append(line)


        texte = "\n".join(ligne) 

        #Supprime tout ce qui est entre crochets
        texte = re.sub(r"\[.*?\]", "", texte)
        #Remplace les multiples sauts de ligne consécutifs
        texte = re.sub(r"\n{2,}", "\n\n", texte).strip()

        #Cherche à nouveau une occ entre crochets
        match = re.search(r"\[.*?\]", texte)
        if match:
            texte = texte[match.start():].lstrip("\n ")

        #Coupe le texte en enlevant les espaces
        texte = texte.strip()         

        #Dictionnaire pour enlever le nbr de phrase nécessaire 
        lines_r = {
            "Eminem": 2,
            "Hozier": 2,
            "Pharrell Williams": 2,
            "Nicky Youre": 0,
            "We the Kings": 0
        }

        #Pour le reste juste 1 phrase
        n = lines_r.get(name, 1)

        #Coupe le ligne en texte
        lignes= texte.split('\n')
        if len(lignes)> n:
            texte = '\n'.join(lignes[n:])

        if not texte:
            print(f"Aucune Parole")
            continue

        #Création du dossier
        namedir = base_path / name
        namedir.mkdir(parents=True, exist_ok=True)

        #Nettoyage du nom sans espace
        artistname = name.replace(" ","")
        filename = f"MCleande{artistname}.txt"
        filepath = namedir / filename

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(texte)

        print(f" C'est good pour {filepath}")

    except Exception as e:
        print(f"Erreur : {e}")    

Scraping Ali Gatie depuis https://genius.com/Ali-gatie-its-you-lyrics...
 C'est good pour C:\Users\AyaKu\PROJECT\data\clean\Ali Gatie\MCleandeAliGatie.txt
Scraping We the Kings depuis https://genius.com/We-the-kings-sad-song-lyrics...
 C'est good pour C:\Users\AyaKu\PROJECT\data\clean\We the Kings\MCleandeWetheKings.txt
Scraping Eminem depuis https://genius.com/Eminem-lose-yourself-lyrics...
 C'est good pour C:\Users\AyaKu\PROJECT\data\clean\Eminem\MCleandeEminem.txt
Scraping Hozier depuis https://genius.com/Hozier-take-me-to-church-lyrics...
 C'est good pour C:\Users\AyaKu\PROJECT\data\clean\Hozier\MCleandeHozier.txt
Scraping Pharrell Williams depuis https://genius.com/Pharrell-williams-happy-lyrics...
 C'est good pour C:\Users\AyaKu\PROJECT\data\clean\Pharrell Williams\MCleandePharrellWilliams.txt
Scraping OneRepublic depuis https://genius.com/Onerepublic-counting-stars-lyrics...
 C'est good pour C:\Users\AyaKu\PROJECT\data\clean\OneRepublic\MCleandeOneRepublic.txt
Scraping Miley Cyr